# OncoBLADE DEMO


In this notebook, a demo of OncoBLADE is provided.

In this demo we will use a subset of the Non Small Cell Lung Carcinoma (NSCLC) samples from the manuscript to show:
1) Cell fraction estimation
2) Cell-type specific gene expression estimation
3) Application of 2

### Move to the right directory and load necessary modules


In [1]:
import sys, os
os.chdir('..') ## Set directory to one up
from Deconvolution.OncoBLADE import Framework_Iterative
from Deconvolution.OncoBLADE import Purify_AllGenes
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
from matplotlib import pyplot as plt
import seaborn as sns

## Creation of a single cell RNAseq signature

OncoBLADE uses a single cell RNAseq (scRNAseq) signature for its bulk RNA deconvolution.
Ideally the scRNAseq is from the same type of tumor as the bulk RNAseq, as is the case here with NSCLC.

To create the signature we use the phenotyped scRNAseq to calculate a mean (Mu) and expected gene expression variability (Omega) per gene per celltype. The expected gene expression variability was estimated by fitting a LOWESS curve to the mean-variance trend measured in the scRNAseq data using the fitTrendVar function implemented in scran.

For this demo we use the same signature as was used in the NSCLC in silico experiments of the manuscript.

In [2]:
# Load NSCLC signature
with open('data/NewSignature_matrices_final_celltype_k1000.pickle', "rb") as infile:
    signature = pickle.load(infile)

## Extract Mu & Omega (nGenes x nCelltypes)
Mu = pd.DataFrame(signature['Mu'], index = signature['Genelist'], columns = signature['celltype_list'])
Omega = pd.DataFrame(signature['Omega'], index = signature['Genelist'], columns = signature['celltype_list'])

## Add pseudocount to Omega as 0 will otherwise give computational problems
Omega = Omega + 0.01
## Print first 5 rows
print(Mu.head())
Omega.head()

        Cancer_cell  Lung_endothelial_cell  Fibroblast  Macrophage  \
A1BG       0.022682               0.072445    0.147478    0.065907   
A2M        0.043105               2.459644    1.926918    0.939422   
A4GALT     0.201694               0.187970    0.162045    0.005898   
AAAS       0.098043               0.058089    0.082713    0.055850   
AACS       0.097181               0.028963    0.043312    0.025011   

        Plasma_cell  Lung_epithelial_cell  CD4_proliferating_T_cell  Monocyte  \
A1BG       0.112361              0.056572                  0.066149  0.116034   
A2M        0.051905              0.045449                  0.046703  0.150999   
A4GALT     0.006781              0.104433                  0.007211  0.004006   
AAAS       0.052889              0.086882                  0.102814  0.027392   
AACS       0.026961              0.053326                  0.032932  0.013841   

        Mast_cell        DC  CD8_effector_T_cell    B_cell  Neutrophil  \
A1BG     0.191235 

,Cancer_cell,Lung_endothelial_cell,Fibroblast,Macrophage,Plasma_cell,Lung_epithelial_cell,CD4_proliferating_T_cell,Monocyte,Mast_cell,DC,CD8_effector_T_cell,B_cell,Neutrophil,Alveolar_cell,CD4_Th17_like_cell,CD4_Treg,NK_cell,CD4_naive_T_cell,CD8_exhausted_T_cell
A1BG,0.036189,0.098136,0.181896,0.080758,0.155971,0.071124,0.088225,0.161703,0.270806,0.162115,0.208026,0.197206,0.010000,0.035627,0.230141,0.223709,0.098819,0.184399,0.219604
A2M,0.059641,1.187655,1.226067,0.803350,0.078141,0.059110,0.065231,0.205610,0.061031,0.324161,0.088831,0.064017,0.076573,0.048453,0.084862,0.059597,0.074665,0.040504,0.065863
A4GALT,0.236096,0.233397,0.198340,0.016346,0.018961,0.122678,0.018528,0.015260,0.016052,0.012545,0.012126,0.037782,0.013378,0.017474,0.015360,0.018494,0.010000,0.010643,0.010000
AAAS,0.121755,0.080672,0.107298,0.069993,0.079423,0.103837,0.131474,0.045965,0.064057,0.074844,0.068537,0.055660,0.021060,0.108113,0.063866,0.084168,0.067927,0.044791,0.071892
AACS,0.120793,0.045239,0.060974,0.036898,0.045523,0.067618,0.048946,0.028173,0.042812,0.037925,0.041852,0.029637,0.016882,0.080934,0.034362,0.042344,0.054352,0.040977,0.036088


## Preparation of Bulk RNAseq and expected tumor fraction
The main innovation of OncoBLADE with respect to other RNA deconvolution methods, is that it allows the user to inform it with cell fraction estimates. OncoBLADE can for example be informed by DNA-derived malignant fraction estimates, making the rest of the deconvolution task significantly easier.
Besides malignant cell fraction estimates, you can also include information on other cell types if you have it. For example if you have Immunohistochemistry stainings on your samples.

Below we show how to prepare your prior fraction expectation and the bulk RNAseq before performing the deconvolution.


In [3]:
## Load in bulk RNAseq, first 5 patients are LUAD, last 5 are LUSC
bulk = pd.read_csv('data/Transcriptome_matrix_subset.txt', sep = '\t')

## Normalize bulk RNAseq to the same scale as the RNAseq, we use counts per 10k.
## Do not Log normalize, this is done within OncoBLADE
samples = [s for s in bulk.columns if 'TCGA' in s]
bulk[samples] = bulk[samples].apply(lambda x: x / sum(x) * 10000)

# set index to gene symbols and put normalized bulk in Y
Y =  bulk.set_index('symbol')
Y.head()


,TCGA-91-6840,TCGA-78-8655,TCGA-NJ-A7XG,TCGA-38-4625,TCGA-38-4627,TCGA-85-A4QQ,TCGA-56-8628,TCGA-63-A5M9,TCGA-77-8128,TCGA-34-5231
symbol,,,,,,,,,,
TSPAN6,1.643189,1.005787,0.986710,1.015124,1.559100,0.308683,0.829310,0.904077,0.727355,0.653832
TNMD,0.001019,0.000000,0.000000,0.000000,0.000437,0.000000,0.000496,0.000000,0.000000,0.000099
DPM1,0.478754,0.269087,0.275330,0.673312,0.293269,0.452470,0.283463,0.516275,0.456260,0.378602
SCYL3,0.185521,0.295398,0.194395,0.117442,0.105425,0.106066,0.144088,0.172375,0.179041,0.200384
C1orf112,0.148485,0.084015,0.056444,0.227574,0.054897,0.111430,0.073408,0.320247,0.160673,0.174342


In [4]:
## Prepare prior expectation
# Load ACE tumor fraction estimates
purities = pd.read_csv('data/ACE_Tumor_purities_squaremodel.tsv', sep = "\t")

# obtain list of expected tumor purities for the 10 samples
expected_tumor_purity =pd.merge(pd.DataFrame({'sample': samples }),purities, how = 'left').ACE.tolist()

# Intialize Expectation (Nsample x Ncell with None for non-tumor celltypes)
Expectation = np.zeros((len(samples), len(signature['celltype_list']))) + np.nan

# iterate over samples
for i in range(len(samples)):
    # iterate over celltypes
    for j,celltype in enumerate(signature['celltype_list']):
        if celltype in ['Cancer_cell', 'Tumor cell']:
            # fetch true tumor purity and add to array
            Expectation[i,j] = expected_tumor_purity[i]
        else:
            pass

print(Expectation)

[[0.55  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.53  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.43  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.2   nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.2   nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.28  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.41  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.64  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.43  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]
 [0.44  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan]]

## Subset signature & bulk on selected genes for deconvolution

OncoBLADE like other RNA deconvolution tools uses a selection of genes for its deconvolution due to the otherwise enormous complexity.

For this we want genes discrimating the cell types which are to be deconvoluted. We use AutoGeneS for this and use the genes it selected on our scRNAseq here.

In [5]:
## Load autogenes which was saved in the signature
AutoGeneS = signature['AutoGeneS']['Unnamed: 0'].tolist()

## Find common AutoGeneS with the bulk data
common_AutoGeneS = [value for value in AutoGeneS if value in Y.index.to_list()]

## Subset Mu, Omega & bulk on AutoGeneS
Mu_AutoGeneS = Mu.loc[common_AutoGeneS,]
Omega_AutoGeneS = Omega.loc[common_AutoGeneS,]
Y_AutoGeneS = Y.loc[common_AutoGeneS,]
Y_AutoGeneS

,TCGA-91-6840,TCGA-78-8655,TCGA-NJ-A7XG,TCGA-38-4625,TCGA-38-4627,TCGA-85-A4QQ,TCGA-56-8628,TCGA-63-A5M9,TCGA-77-8128,TCGA-34-5231
symbol,,,,,,,,,,
A2M,10.682426,10.446552,1.925969,5.821721,23.110129,0.628959,13.675426,3.468265,2.097612,3.076826
ABCA1,0.746502,0.447581,0.163208,0.999605,0.387482,0.533274,0.677782,0.248778,0.751002,0.190345
ABCA3,0.451231,1.060800,3.984150,1.075697,0.364474,0.055542,4.889802,0.343559,0.237948,0.234378
AC022182.3,0.001019,0.000598,0.000383,0.000801,0.001747,0.000173,0.002480,0.001531,0.000000,0.000795
AC133644.2,0.008155,0.011660,0.001913,0.007209,0.016455,0.002768,0.003968,0.008168,0.008023,0.004373
...,...,...,...,...,...,...,...,...,...,...
ZMIZ1,2.380857,0.627272,0.478144,0.497400,0.909657,1.252553,1.273724,0.392907,1.153000,1.295239
ZMYND10,0.090382,0.008073,0.064288,0.010513,0.078923,0.079593,0.048360,0.004594,0.007390,0.048804
ZNF276,0.193336,0.240683,0.235915,0.076893,0.039753,0.243278,0.412919,0.118093,0.164051,0.218176


## Setting the parameters for OncoBLADE
Before running OncoBLADE there is one more thing to do, setting the (hyper)parameters.
The key parameters used in OncoBLADE are:
- Hyperparameters (`hyperpars`): `Alpha`, `Alpha0`, `Kappa0` and `SigmaY`.
- `Nrep`: Number of repeats for evaluating each parameter configuration.
- `Nrepfinal`: Number of repeated optimizations for the final parameter set.
- `Njob`: Number of parallel jobs.

In [6]:
pars = {
    'Alpha': 1,
    'Alpha0': 1000,
    'Kappa0': 1,
    'SY': 1,
    'Nrep': 3, ## small for demo purposes, for real application >10 is advised
    'Nrepfinal': 100, ## small for demo purposes, for real application >1000 is advised
    'Njob': 4 ## Number of parallel jobs.
}


## 1) Cell fraction estimation by OncoBLADE
Now we are ready to run the first step of OncoBLADE, the cell fraction estimation. Here it is applied to 10 NSCLC samples.

OncoBLADE produce several outcomes:
- `final_obj`: final BLADE object with optimized variational parameters
- `best_obj`: BLADE object trained with the best parameter set found by the Empirical Bayes framework. Empirical Bayes framework is applied after selecting a subset of samples (5 samples; indicated by `Ind_sample` below), and thus the outcome contains only 5 samples. If `Nsample` <= 5, `final_obj` is identical to `best_obj`.
- `best_set`: Best parameter set defined by Empirical Bayes framework.
- `outs`: Outcome of BLADE for every possible combination of hyperparameters, used in the Empirical Bayes framework. 


In [7]:
final_obj, best_obj, best_set, outs = Framework_Iterative(
            Mu_AutoGeneS.to_numpy(), Omega_AutoGeneS.to_numpy(),Y_AutoGeneS.to_numpy(),
            Alpha=pars['Alpha'], Alpha0=pars['Alpha0'], 
            Kappa0=pars['Kappa0'], sY = pars['SY'],
            Nrep= pars['Nrep'], Njob= pars['Njob'], IterMax= pars['Nrepfinal'], Expectation = Expectation)

## Save BLADE output in one dictionary
BLADE_output = {
    'final_obj': final_obj,
    'best_obj': best_obj,
    'best_set': best_set,
    'outs' : outs,
    'genes' : common_AutoGeneS
     }


all of 548 genes are used for optimization.
Initialization with Support vector regression


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:    2.4s remaining:    2.4s
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:    2.7s remaining:    1.2s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    3.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:871: Warning: No optimization is not done yet
  obj.Check_health()
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:871: Warning: No optimization is not done yet
  obj.Check_health()
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:871: Warning: No optimization is not done yet
  obj.Check_health()


No feature filtering is done (fsel = 0)


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed: 13.8min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed: 13.8min finished


## 2) Cell type specific gene expression estimation by OncoBLADE
After step 1, we can fix the estimated cell fractions which allows us to estimate the cell type specific gene expression in parallel per gene. Here we estimate the cell type specific gene expression levels for the top 1819 highly variable genes in the scRNAseq data.


In [ ]:
## Load autogenes which was saved in the signature
hvgenes = signature['GeneList_hvg']

## Find common hvg with the bulk data
common_hvgenes = [value for value in hvgenes if value in Y.index.to_list()]

## Subset Mu, Omega & bulk on AutoGeneS
Mu_hvgenes = Mu.loc[common_hvgenes,]
Omega_hvgenes = Omega.loc[common_hvgenes,]
Y_hvgenes = Y.loc[common_hvgenes,]

## Estimate cell type specific gene expression
final_obj_2, obj_func = Purify_AllGenes(BLADE_output, Mu_hvgenes.to_numpy(), Omega_hvgenes.to_numpy(), Y_hvgenes.to_numpy(), pars['Njob'])



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:974: Warning: No optimization is not done yet
  obj.Check_health()
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:974: Warning: No optimization is not done yet
  obj.Check_health()
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:974: Warning: No optimization is not done yet
  obj.Check_health()
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    3.9s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    5.1s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    7.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.5s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   13.2s
/net/beegfs/cfg/tgac/m.steketee/OncoBLADE/Deconvolution/OncoBLADE.py:974: Warning: No opti